# One-class SVM における誤検出率（nu=0.01に変更）

scikit-learn の <b>One-class SVM</b> の、誤検出率（アノマリーでないデータがアノマリーと検出されてしまう率）を求める方法の検討になります。


## (0) 確認結果

結果から申し上げると、誤検出率は、パラメータの nu 値（0.01＝外れ判定される件数を全体の 1% と想定）を上回る結果（0.041）となりました。

One-class SVMが元来（１件もアノマリーデータが混入していない訓練データを使用して学習したにもかかわらず）訓練データ中に常に「一定数のアノマリーが存在している」前提のモデルなので、一定の誤検出率が上がってしまうのは、不可避と考えております。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_septeni_conversation.csv'
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/23 PM 09:40:51 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 28272
2017/03/23 PM 09:40:51 TextArray#__init__ start
2017/03/23 PM 09:41:05 TextArray#to_vec start
2017/03/23 PM 09:41:05 TextArray#to_vec end


In [5]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=28272, feature=2748


### (2-2) 外れデータのTF-IDFベクター

<b><a href="12.ipynb">こちらで再検証した時と同じ質問文</a></b>をつかっております。

In [6]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
#from learning.core.training_set.text_array import TextArray
from prototype.modules.text_array import TextArray

test_X = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない
]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/23 PM 09:41:05 TextArray#__init__ start
2017/03/23 PM 09:41:05 TextArray#to_vec start
2017/03/23 PM 09:41:05 TextArray#to_vec end


In [7]:
vocabulary = text_array._vectorizer.vocabulary_
dumped_features = TestTool.get_dumped_features(X_error, vocabulary)
for d in dumped_features:
    print(d)

index=0[]
index=1[]
index=2[調達=1.000]
index=3[何=1.000]
index=4[いい=0.707 何=0.707]
index=5[いい=0.707 何=0.707]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.577 会社=0.577 誰=0.577]
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
index=9[休暇=0.500 取る=0.500 海外=0.500 行く=0.500]
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447]


## (3) One-class SVMで学習

線形カーネルを使用しています。

また、パラメータの nu を 0.01 へ変更しています。

In [8]:
'''
    One-class SVM with linear kernel, anomary is half
        fn(i,j) = i'*j
'''
from sklearn import svm
clf_linear = svm.OneClassSVM(
    kernel='linear', # linearカーネルを使用
    nu=0.01          # 外れ判定される件数の見積もりを全体の 1% と想定
    ) 
clf_linear.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.01, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

## (4) 誤検出率の算定

### (4-1) 訓練データを使って予測実行

アノマリーでないデータ＝訓練データを使って予測した時、アノマリーと検出されてしまうデータがあるかどうか確認します。

In [9]:
'''
    訓練データを使用して、外れ判定を実行してみます
'''
pred_by_linear_for_train = clf_linear.predict(X)
pred_by_linear_for_train

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

### (4-2) 誤検出率を算出

誤検出率＝アノマリー検出件数 ÷ 訓練データ件数 になります。

パラメータの nu 値（0.01）を上回る結果となりました。

In [10]:
n_anomaly = len([c for c in pred_by_linear_for_train if c == -1])

print('誤検出率=%0.3f [サンプル件数=%d, アノマリー件数=%d]' % (n_anomaly / n_sample, n_sample, n_anomaly))

誤検出率=0.041 [サンプル件数=28272, アノマリー件数=1155]


### (4-3) 参考：アノマリーと検出された訓練データだけを抽出

In [11]:
'''
    (4-1) において、アノマリーと検出されたデータを抽出
'''
from scipy.sparse import lil_matrix, csr_matrix

X_misdetected_temp = lil_matrix((n_anomaly, n_feature))
idx = 0
for i, v in enumerate(pred_by_linear_for_train):
    if v == -1:
        X_misdetected_temp[idx] = X.getrow(i)
        idx += 1

### (4-4) 参考：アノマリーと検出された訓練データの特徴を確認

長いですが、ご参考までに全量表示します。

例えば「ゴミを捨てたいのですが？」は回答不能になってしまうという事になります。

In [12]:
dumped_features = TestTool.get_dumped_features(X_misdetected_temp, vocabulary)
for d in dumped_features:
    print(d)

index=0[suica=0.707 登録=0.707]
index=1[パスモ=0.707 登録=0.707]
index=2[icoca=0.707 登録=0.707]
index=3[ゴミ=0.707 捨てる=0.707]
index=4[割れ物=0.707 捨てる=0.707]
index=5[ドライバー=0.707 借りる=0.707]
index=6[ほしい=0.707 ホワイトボードマーカー=0.707]
index=7[消しゴム=0.707 無くなる=0.707]
index=8[デート=0.577 印=0.577 壊れる=0.577]
index=9[ク=0.577 ネットワ=0.577 繋がる=0.577]
index=10[lan=0.577 ケーブル=0.577 借りる=0.577]
index=11[dat=0.577 winmail=0.577 解凍=0.577]
index=12[プリンターー=0.707 換える=0.707]
index=13[プリンターー=0.707 換える=0.707]
index=14[印鑑=0.707 捺印=0.707]
index=15[印鑑=0.707 捺印=0.707]
index=16[印鑑=0.707 捺印=0.707]
index=17[印紙=0.500 収入=0.500 欲しい=0.500 説明=0.500]
index=18[契約=0.577 書=0.577 見せる=0.577]
index=19[やり方=0.577 小口=0.577 申請=0.577]
index=20[プリンター=0.707 他=0.707]
index=21[コピー=0.707 白黒=0.707]
index=22[イントラ=0.707 見る=0.707]
index=23[予定=0.577 変える=0.577 座席=0.577]
index=24[予定=0.577 変える=0.577 座席=0.577]
index=25[カードキーケース=0.707 下さる=0.707]
index=26[リザーブ=0.577 ルーム=0.577 ワーク=0.577]
index=27[室=0.577 押さえる=0.577 顧問=0.577]
index=28[涼しい=0.707 部屋=0.707]
index=29[下げる=0.50

## (5) 参考：外れデータを使って予測

頻出ワードが含まれる質問文に対しては、アノマリーと検出されない傾向があります。

ただし、明らかにminorityまたは異常とみなされる質問文に対しては、アノマリーと検出するようです。

In [13]:
'''
    外れデータを使用して、外れ判定を実行してみます
'''
pred_by_linear_for_error = clf_linear.predict(X_error)
vocabulary = text_array._vectorizer.vocabulary_

dumped_features = TestTool.get_dumped_features(X_error, vocabulary)
for k, d in enumerate(dumped_features):
    if pred_by_linear_for_error[k] == -1:
        print(d, '--->detected as anomaly')
    else:
        print(d)

index=0[] --->detected as anomaly
index=1[] --->detected as anomaly
index=2[調達=1.000] --->detected as anomaly
index=3[何=1.000] --->detected as anomaly
index=4[いい=0.707 何=0.707]
index=5[いい=0.707 何=0.707]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.577 会社=0.577 誰=0.577] --->detected as anomaly
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
index=9[休暇=0.500 取る=0.500 海外=0.500 行く=0.500] --->detected as anomaly
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447] --->detected as anomaly


### (5-2) 参考：頻出ワード（100位まで）

In [14]:
ext_feature_count = [0.0 for i in range(n_feature)]
tot_feature_count = 0.0
for i in range(n_sample):
    
    arr = X[i].toarray()[0]
    for j in range(n_feature):
        if arr[j] == 0.0:
            continue
        ext_feature_count[j] += 1.0
        tot_feature_count += 1.0
        
ext_feature_list = []
for k, v in enumerate(ext_feature_count):
    item = TestTool.get_item_from_vocabulary(vocabulary, k)
    elem = (item, v, v / tot_feature_count)
    ext_feature_list.append(elem)

sorted_list = sorted(ext_feature_list, key=lambda x: x[1], reverse=True)

In [15]:
for i in range(100):
    print("[%s] count=%d, rate=%0.4f" % sorted_list[i])

[メール] count=4900, rate=0.0263
[できる] count=3516, rate=0.0189
[れる] count=3379, rate=0.0181
[2010] count=2694, rate=0.0145
[outlook] count=2477, rate=0.0133
[教える] count=2394, rate=0.0129
[暗号] count=2252, rate=0.0121
[設定] count=2100, rate=0.0113
[表示] count=1973, rate=0.0106
[方法] count=1856, rate=0.0100
[e] count=1769, rate=0.0095
[インストール] count=1766, rate=0.0095
[管理] count=1720, rate=0.0092
[利用] count=1624, rate=0.0087
[どう] count=1508, rate=0.0081
[ツール] count=1506, rate=0.0081
[化] count=1414, rate=0.0076
[システム] count=1408, rate=0.0076
[dki] count=1404, rate=0.0075
[it] count=1328, rate=0.0071
[知る] count=1319, rate=0.0071
[資産] count=1302, rate=0.0070
[パスワード] count=1262, rate=0.0068
[なる] count=1236, rate=0.0066
[使用] count=1234, rate=0.0066
[pointsec] count=1226, rate=0.0066
[いい] count=1221, rate=0.0066
[端末] count=1050, rate=0.0056
[可能] count=1006, rate=0.0054
[ファイル] count=1002, rate=0.0054
[欲しい] count=994, rate=0.0053
[出来る] count=968, rate=0.0052
[アウト] count=932, rate=0.0050
[先生] count=914, 